# Fine tuning for spam classification

This follows chapter 6 of the book [Build a Large Language Model (From Scratch)](https://www.manning.com/books/build-a-large-language-model-from-scratch).

In [ ]:
import import_ipynb
import openai # type:ignore
import gpt # type:ignore
import pandas as pd
import urllib.request
import ssl
import zipfile
import os
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
import tiktoken

## Download and preprocess the UCI spam data

The fine folks at the University of California at Irvine have provided a nice little data set for SMS spam.
Let's download that and save it in a convenient CSV format.

In [30]:
url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

# NOTE: on 6/21/25, the UCI archive server is unreachable. I downloaded this file
# manually from a mirror.
def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
    if data_file_path.exists():
        print(f"{data_file_path} already exists. Skipping download and extraction.")
        return
     
    ssl_context = ssl._create_unverified_context()

    with urllib.request.urlopen(url, context=ssl_context) as response:
        with open(zip_path, "wb") as out_file:
            out_file.write(response.read())
    
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_path)

    original_file_path = Path(extracted_path) / "SMSSpamCollection"
    os.rename(original_file_path, data_file_path)    
    print(f"File downloaded and saved as {data_file_path}")


The data set contains 4825 ham messages and only 747 spam messages. Since we want an equal number of both, we'll have to take 747 ham messages at random and discard the rest.

In [31]:
def create_balanced_dataset(df):
    df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"])
    num_spam = df[df["Label"] == "spam"].shape[0]
    ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)
    balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])
    return balanced_df

Now we want to create the following splits:
- 70% for training
- 10% for validation
- 20% for testing

In [32]:
def random_split(df, train_frac, validation_frac):
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)

    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)

    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]

    return train_df, validation_df, test_df

In [33]:
def save_csv():
    download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)
    df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"])
    balanced_df = create_balanced_dataset(df)
    balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})
    train_df, validation_df, test_df = random_split(balanced_df, 0.7, 0.1)
    train_df.to_csv("train.csv", index=None) # type:ignore
    validation_df.to_csv("validation.csv", index=None) # type:ignore
    test_df.to_csv("test.csv", index=None) # type:ignore

# Uncomment if you haven't saved this yet
# save_csv()

## SpamDataset

This class:
1. Pre-tokenizes the texts from the dataset
2. Truncates any sequences that are longer than the maximum length (or the longest text, if no maximum is set).
3. Pads any sequences shorter than the max length.

In [38]:
class SpamDataset(Dataset):
    def __init__(self, csv_file: Path, tokenizer: tiktoken.Encoding, max_length:int|None=None, pad_token_id:int=50256):
        self.data = pd.read_csv(csv_file)

        # Pre-tokenize texts
        self.encoded_texts = [
            tokenizer.encode(text) for text in self.data["Text"]
        ]

        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
        
            # truncate sequences that are longer than max_length
            self.encoded_texts = [
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]

        # pad the sequences
        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]

    def __getitem__(self, idx):
        encoded = self.encoded_texts[idx]
        label = self.data.iloc[idx]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        return max([len(txt) for txt in self.encoded_texts])

In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")
train_dataset = SpamDataset(
    csv_file=Path("train.csv"),
    tokenizer=tokenizer,
)
max_length = train_dataset.max_length
val_dataset = SpamDataset(
    csv_file=Path("validation.csv"),
    max_length=max_length,
    tokenizer=tokenizer,
)
test_dataset = SpamDataset(
    csv_file=Path("test.csv"),
    max_length=max_length,
    tokenizer=tokenizer,
)

120


In [ ]:
num_workers = 0
batch_size = 8

gpt.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

# Note: I think drop_last is true because the last batch won't have enough messages in it,
# not because the last message might be short.